In [19]:
import time
import pandas as pd
import geopandas as gpd
from geopandas.tools import geocode
import geopy

In [4]:
address = "Mercedes-Benz Museum"
result = geocode(address, provider='nominatim',)
print(result.geometry.iloc[0].x, ';', result.geometry.iloc[0].y)
result

9.234112312115384 ; 48.788188250000005


,geometry,address
0,POINT (9.23411 48.78819),"Mercedes-Benz Museum, 100, Mercedesstraße, Was..."


### University data geocoding

In [25]:
universities = pd.read_csv('./data/top_universities.csv')
universities.head()

,Name
0,University of Oxford
1,University of Cambridge
2,Imperial College London
3,ETH Zurich
4,UCL


In [26]:
def geocode_func(name):
    try:        
        result = geocode(name, provider='nominatim')
        location = result.geometry.iloc[0]
        time.sleep(1)
        return pd.Series({'Latitude': location.x, 'Longitude': location.y,
                          'Address': result.address.iloc[0], 'geometry': location})
    except Exception as e:
        print(name, ':', e)
        return None

universities[['Latitude', 'Longitude', 'Address', 'geometry']] = universities.apply(lambda x: geocode_func(x['Name']), axis=1)
universities.head(90)

In [20]:
geopy.geocoders.options.default_user_agent = "university-geo-code"

### Europe Boundaries

In [16]:
worldfilepath = gpd.datasets.get_path('naturalearth_lowres')
world = gpd.read_file(worldfilepath)

europe = world[world['continent'] == 'Europe']

europe_boundary = europe[["name", "geometry"]]
europe_stats = europe[["name", "pop_est", "gdp_md_est"]]

In [17]:
europe_boundary.head()

,name,geometry
18,Russia,"MULTIPOLYGON (((178.725 71.099, 180.000 71.516..."
21,Norway,"MULTIPOLYGON (((15.143 79.674, 15.523 80.016, ..."
43,France,"MULTIPOLYGON (((-51.658 4.156, -52.249 3.241, ..."
110,Sweden,"POLYGON ((11.027 58.856, 11.468 59.432, 12.300..."
111,Belarus,"POLYGON ((28.177 56.169, 29.230 55.918, 29.372..."


In [18]:
europe_stats.head()

,name,pop_est,gdp_md_est
18,Russia,142257519,3745000.0
21,Norway,5320045,364700.0
43,France,67106161,2699000.0
110,Sweden,9960487,498100.0
111,Belarus,9549747,165400.0
